In [13]:
import pandas as pd

def encode_categorical_values(dataframes):
    student_info = dataframes['student_info']
    student_vle = dataframes['student_vle']
    student_reg = dataframes['student_reg']
    student_assessment = dataframes['student_assessment']
    assessments = dataframes['assessments']
    courses = dataframes['courses']
    vle = dataframes['vle']

    edu_map = {
        'No Formal quals': 0,
        'Lower Than A Level': 1,
        'A Level or Equivalent': 2,
        'HE Qualification': 3,
        'Post Graduate Qualification': 4
    }
    student_info['highest_education'] = student_info['highest_education'].map(edu_map)
    
    age_map = {
        '0-35': 0,
        '35-55': 1,
        '55<=': 2
    }
    student_info['age_band'] = student_info['age_band'].map(age_map)
    
    imd_map = {
        '0-10%': 0,
        '10-20%': 1,
        '20-30%': 2,
        '30-40%': 3,
        '40-50%': 4,
        '50-60%': 5,
        '60-70%': 6,
        '70-80%': 7,
        '80-90%': 8,
        '90-100%': 9,
        'Missing': -1,
    }
    student_info['imd_band'] = student_info['imd_band'].map(imd_map) 
      
    student_info_encoded = pd.get_dummies(student_info, columns=['gender', 'region', 'disability'], drop_first=True)
    assessments_encoded = pd.get_dummies(assessments, columns=['assessment_type'], drop_first=True)
    vle_encoded = pd.get_dummies(vle, columns=['activity_type'], drop_first=True)


    return {
        "student_info": student_info_encoded,
        "student_vle": student_vle,
        "student_reg": student_reg,
        "student_assessment": student_assessment,
        "assessments": assessments_encoded,
        "courses": courses,
        "vle": vle_encoded
    }